# Testing custom CapsNet trained on padded and translated MNIST train set on affNIST test set

In [2]:
from keras import backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Conv2D, Activation, Dense, Dropout, Lambda
from keras.layers import BatchNormalization, MaxPooling2D, Flatten, Conv1D
from convcaps.capslayers import ConvertToCaps, Conv2DCaps, FlattenCaps
from convcaps.capslayers import DenseCaps, CapsToScalars
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard
from keras import optimizers
from keras import regularizers
from keras import losses
import numpy as np
from scipy.io import matlab
import os

Using TensorFlow backend.


## Load train set generated in generate_datasets.py

In [3]:
img_rows, img_cols = 40, 40
num_classes = 10

x_train = np.load('generateddatasets/x_train_only_translation.npy').astype(np.float32)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train /= 255.0

y_train = np.load('generateddatasets/y_train_only_translation.npy')
y_train = to_categorical(y_train, num_classes)

input_shape = (img_rows, img_cols, 1)

## Assemble CapsNet

In [4]:
l2 = regularizers.l2(l=0.001)

inp = Input(shape=input_shape)
l = inp

l = Conv2D(16, (5, 5), strides=(2, 2), activation='relu', kernel_regularizer=l2)(l)  # common conv layer
l = BatchNormalization()(l)
l = ConvertToCaps()(l)
l = Conv2DCaps(6, 4, (3, 3), (2, 2), r_num=1, b_alphas=[1, 1, 1], kernel_regularizer=l2)(l)
l = Conv2DCaps(5, 5, (3, 3), (1, 1), r_num=1, b_alphas=[1, 1, 1], kernel_regularizer=l2)(l)
l = Conv2DCaps(4, 6, (3, 3), (1, 1), r_num=1, b_alphas=[1, 1, 1], kernel_regularizer=l2)(l)
l = Conv2DCaps(3, 7, (3, 3), (1, 1), r_num=1, b_alphas=[1, 1, 1], kernel_regularizer=l2)(l)

l = FlattenCaps()(l)  # transform to a dense caps layer
l = DenseCaps(10, 8, r_num=3, b_alphas=[1, 8, 8], kernel_regularizer=l2)(l)
l = CapsToScalars()(l)

model = Model(inputs=inp, outputs=l, name='40x40_input_capsnet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 16)        416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 18, 18, 16)        64        
_________________________________________________________________
convert_to_caps_1 (ConvertTo (None, 18, 18, 16, 1)     0         
_________________________________________________________________
conv2d_caps_1 (Conv2DCaps)   (None, 8, 8, 6, 4)        3456      
_________________________________________________________________
conv2d_caps_2 (Conv2DCaps)   (None, 6, 6, 5, 5)        5400      
_________________________________________________________________
conv2d_caps_3 (Conv2DCaps)   (None, 4, 4, 4, 6)        5400      
__________

In [5]:
def caps_objective(y_true, y_pred):
    return K.sum(y_true * K.clip(0.8 - y_pred, 0, 1) ** 2 + 0.5 * (1 - y_true) * K.clip(y_pred - 0.3, 0, 1) ** 2)

optimizer = optimizers.Adam(lr=0.001)
model.compile(loss=caps_objective,
              optimizer=optimizer,
              metrics=['accuracy'])

In [6]:
w_path = os.path.join('weights', model.name)

if not os.path.exists(w_path):
    os.makedirs(w_path)
    
f_name = os.path.join(w_path, 'weights.{epoch:02d}.hdf5')    

In [ ]:
# model.load_weights('weights/40x40_input_capsnet/weights.03.hdf5')

# Train for 4 epochs

In [37]:
batch_size = 32
num_epochs = 4

h = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=num_epochs, initial_epoch=0,
          verbose=1, validation_split=0.09,
          callbacks=[ModelCheckpoint(f_name)])

Train on 436800 samples, validate on 43200 samples
Epoch 1/4
436800/436800 [==============================] - 910s - loss: 0.7221 - acc: 0.9490 - val_loss: 0.3835 - val_acc: 0.9764
Epoch 2/4
436800/436800 [==============================] - 925s - loss: 0.3840 - acc: 0.9749 - val_loss: 0.3295 - val_acc: 0.9796
Epoch 3/4
436800/436800 [==============================] - 743s - loss: 0.3569 - acc: 0.9763 - val_loss: 0.3361 - val_acc: 0.9794
Epoch 4/4
436800/436800 [==============================] - 528s - loss: 0.3426 - acc: 0.9772 - val_loss: 0.3072 - val_acc: 0.9796


## Test on affNIST test set

In [8]:
f = matlab.loadmat('affnist/test.mat')

x_test = f['affNISTdata'][0, 0][2].transpose().astype(np.float32)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_test /= 255
input_shape = (img_rows, img_cols, 1)

y_test = f['affNISTdata'][0, 0][4].transpose()

score = model.evaluate(x_test, y_test, verbose=1)
print('')
print('Test score: ', score[0])
print('Test accuracy: ', score[1])    

319936/320000 [============================>.] - ETA: 0s
Test score:  3.78991742519
Test accuracy:  0.704396875
